# **Importing Dependencies**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm

In [ ]:
%pip install opendatasets --quiet
import opendatasets as od
import os

od.download("https://www.kaggle.com/datasets/aseemdandgaval/23-pet-breeds-image-classification")

if not os.path.exists("23-pet-breeds-image-classification"):
  print("The Dataste has been downloaded as {23-pet-breeds-image-classification}")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: arkapravodas03
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/aseemdandgaval/23-pet-breeds-image-classification


100%|██████████| 794M/794M [00:42<00:00, 19.5MB/s]


In [ ]:
%pip install torch --quiet
import torch
import torch.nn as nn
import torch.nn.functional as F

# Checking the version of Pytorch
print(torch.__version__)

# Device Agnostic Code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"The device being used is: {device}")

2.5.1+cu121
The device being used is: cuda


In [ ]:
BREEDS_DIR = "/content/23-pet-breeds-image-classification/Pet_Breeds"

breeds = [breed for breed in os.listdir(BREEDS_DIR)]
print(f"The number of pet breeds in the dataset: {len(breeds)}")

breeds

The number of pet breeds in the dataset: 23


['maine coon',
 'sphynx',
 'abyssinian',
 'golden retriever',
 'mumbai cat',
 'persian cat',
 'corgi',
 'pug',
 'boxer',
 'chihuahua',
 'ragdoll cat',
 'american shorthair',
 'labrador',
 'beagle',
 'husky',
 'german shepherd',
 'pomeranian',
 'rottwiler',
 'yorkshire terrier',
 'siamese cat',
 'bulldog',
 'shiba inu',
 'dachshund']

In [ ]:
data = []

for breed in breeds:
  breed_path = os.path.join(BREEDS_DIR, breed)
  for image in os.listdir(breed_path):
    image_path = os.path.join(breed_path, image)
    # Appending Img_path, Specie to a List
    data.append([
        image_path,
        breed
        ])

# Converting the List to a Data Frame
data = pd.DataFrame(data, columns=["image_path", "breed"])
data.head(10)

,image_path,breed
0,/content/23-pet-breeds-image-classification/Pe...,maine coon
1,/content/23-pet-breeds-image-classification/Pe...,maine coon
2,/content/23-pet-breeds-image-classification/Pe...,maine coon
3,/content/23-pet-breeds-image-classification/Pe...,maine coon
4,/content/23-pet-breeds-image-classification/Pe...,maine coon
5,/content/23-pet-breeds-image-classification/Pe...,maine coon
6,/content/23-pet-breeds-image-classification/Pe...,maine coon
7,/content/23-pet-breeds-image-classification/Pe...,maine coon
8,/content/23-pet-breeds-image-classification/Pe...,maine coon
9,/content/23-pet-breeds-image-classification/Pe...,maine coon


In [ ]:
for breed in breeds:
  no_of_img = data['breed'][data['breed'] == breed].count() # Filter the 'breed' column by the current breed and get the count
  print(f"The number of images of breed {breed} is {no_of_img}")

The number of images of breed maine coon is 170
The number of images of breed sphynx is 170
The number of images of breed abyssinian is 170
The number of images of breed golden retriever is 170
The number of images of breed mumbai cat is 141
The number of images of breed persian cat is 170
The number of images of breed corgi is 170
The number of images of breed pug is 170
The number of images of breed boxer is 170
The number of images of breed chihuahua is 170
The number of images of breed ragdoll cat is 170
The number of images of breed american shorthair is 170
The number of images of breed labrador is 170
The number of images of breed beagle is 170
The number of images of breed husky is 170
The number of images of breed german shepherd is 170
The number of images of breed pomeranian is 170
The number of images of breed rottwiler is 170
The number of images of breed yorkshire terrier is 170
The number of images of breed siamese cat is 170
The number of images of breed bulldog is 170


In [ ]:
# Preprocessing and Dataset Class
import cv2
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# Data Preprocessing Function
def preprocess_image(image_path, target_size=(512, 512)):
    image = cv2.imread(image_path)
    # Check if the image was loaded successfully
    if image is None:
        print(f"Error loading image: {image_path}")
        return None  # Return None if image loading failed

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image)


In [ ]:
# Convert the images Tensors
image_tensors = []
for index, row in data.iterrows():
    image_path = row['image_path']
    tensor = preprocess_image(image_path)
    if tensor is not None:  # Check if the image was read successfully
      image_tensors.append(tensor)
    else:
      print(f"Skipping image at {image_path} due to read error.")

print(f"Number of successfully processed image tensors: {len(image_tensors)}")
# Now 'image_tensors' contains a list of tensors, each representing a preprocessed image.

Error loading image: /content/23-pet-breeds-image-classification/Pet_Breeds/mumbai cat/mumbai cat_71.jpg
Skipping image at /content/23-pet-breeds-image-classification/Pet_Breeds/mumbai cat/mumbai cat_71.jpg due to read error.


In [ ]:
# Seeing the Transformed Images
num_img = 9
fig, axes = plt.subplots(3, 3, figsize=(10, 10))
axes = axes.flatten()

for idx, image_tensor in enumerate(image_tensors[:num_img]):
    # Convert the tensor to a NumPy array for display
    image_np = image_tensor.permute(1, 2, 0).numpy()
    # permute is used to change the order of dimensions from (C, H, W) to (H, W, C)
    # where C is the channel, H is the height, and W is the width.
    # This is necessary because Matplotlib expects images in the (H, W, C) format.

    # Un-normalize the image (reverse the normalization applied during preprocessing)
    # image_np = image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    # image_np = np.clip(image_np, 0, 1)  # Clip values to the range [0, 1]

    axes[idx].imshow(image_np)
    axes[idx].axis('off')  # Turn off axis labels

plt.tight_layout()  # Adjust subplot parameters for a tight layout
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score

class Pet_Classification(nn.Module):
  def __init__(self, num_classes):
    super(Pet_Classification, self).__init__()

    self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)

    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.fc1 = nn.Linear(64 * 64 * 64, 128)
    self.fc2 = nn.Linear(128, num_classes)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = x.view(-1, 64 * 64 * 64)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

pet_classifier = Pet_Classification(num_classes=len(breeds))
pet_classifier.to(device)

In [ ]:
# Loss Function
criterion = nn.CrossEntropyLoss()
# Gardient Descent Weight updation
optimizer = torch.optim.Adam(pet_classifier.parameters(), lr=0.001)

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Making Training, Validation and Testing Dataloaders
train_df, test_df = train_test_split(data,
                                     test_size=0.2,
                                     random_state=42,
                                     stratified=data['breed'])

train_df, val_df = train_test_split(train_df,
                                    test_size=0.2,
                                    random_state=42)

train_loader = DataLoader(train_df, batch_size=32, shuffle=True)
val_loader = DataLoader(val_df, batch_size=32)
test_loader = DataLoader(test_df, batch_size=32)

In [ ]:
# Training Loop with Evaluation and Metrics Visualization
num_epochs = 10
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []
train_f1_scores, val_f1_scores = [], []

for epoch in range(num_epochs):
    # Training Phase
    pet_classifier.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    train_labels, train_preds = [], []

    for inputs, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = pet_classifier(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        train_labels.extend(labels.cpu().numpy())
        train_preds.extend(predicted.cpu().numpy())

    train_loss = running_loss / len(train_loader)
    train_accuracy = accuracy_score(train_labels, train_preds)
    train_f1 = f1_score(train_labels, train_preds, average='weighted')

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    train_f1_scores.append(train_f1)

    # Validation Phase
    pet_classifier.eval()
    val_running_loss = 0.0
    val_labels, val_preds = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = pet_classifier(inputs)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(predicted.cpu().numpy())

    val_loss = val_running_loss / len(val_loader)
    val_accuracy = accuracy_score(val_labels, val_preds)
    val_f1 = f1_score(val_labels, val_preds, average='weighted')

    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    val_f1_scores.append(val_f1)

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Train F1: {train_f1:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {val_f1:.4f}')

print("Training finished.")

# Plotting Metrics
epochs_range = range(1, num_epochs + 1)
plt.figure(figsize=(15, 5))

# Loss Plot
plt.subplot(1, 3, 1)
plt.plot(epochs_range, train_losses, label='Train Loss')
plt.plot(epochs_range, val_losses, label='Val Loss')
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy Plot
plt.subplot(1, 3, 2)
plt.plot(epochs_range, train_accuracies, label='Train Accuracy')
plt.plot(epochs_range, val_accuracies, label='Val Accuracy')
plt.title('Accuracy vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# F1-Score Plot
plt.subplot(1, 3, 3)
plt.plot(epochs_range, train_f1_scores, label='Train F1 Score')
plt.plot(epochs_range, val_f1_scores, label='Val F1 Score')
plt.title('F1-Score vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score

# Testing Phase
pet_classifier.eval()
test_loss = 0.0  # Initialize test loss
test_labels, test_preds = [], []

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = pet_classifier(inputs)
        loss = criterion(outputs, labels)  # Calculate loss for this batch
        test_loss += loss.item()  # Accumulate loss

        _, predicted = torch.max(outputs, 1)
        test_labels.extend(labels.cpu().numpy())
        test_preds.extend(predicted.cpu().numpy())

# Calculate metrics
avg_test_loss = test_loss / len(test_loader)  # Average test loss
accuracy = accuracy_score(test_labels, test_preds)
f1 = f1_score(test_labels, test_preds, average='weighted')
precision = precision_score(test_labels, test_preds, average='weighted')

# Print the results
print(f"Test Loss: {avg_test_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")